# Amex - trojans - Version 2

## Schedule
- feature engineering 1.5 months
- model tuning 1 month

This is a starter code template for numerical variables. 

- Time features(the groupby logic) should be created based on the distribution of variables
- If the distribution is kind of wiggle, record it in the dictionary below

##### DDL: 7.7(Meet on 6.30)
##### Deliverables: A csv file/ pickle file + code

Upload Address: https://drive.google.com/drive/folders/1zyNIH8U4atqI8X48IFbeERpX65nVSRH5?usp=sharing

Features' Dictionary: https://docs.google.com/spreadsheets/d/1UsodGz_lkmzVmvweA_0YZZwRoB2VBpdAVuEuPtb38wI/edit?usp=sharing

- Hongyi: P,R,D

- Duanmu: 'D_' top 56

- Ziyi: 'B_'

- Haobo: NN Model

In [1]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, StratifiedKFold
import lightgbm as lgb
import xgboost as xgb
import sklearn

In [2]:
# add data 
# Download: https://www.kaggle.com/datasets/munumbutt/amexfeather
# Or press "add data" and search "amexfeather", you will see AMEX-Feather-Dataset

train = pd.read_feather('../input/amexfeather/train_data.ftr')
test = pd.read_feather('../input/amexfeather/test_data.ftr')

In [3]:
tempTrain = train[['customer_ID','S_2','B_10','target']]
tempTrain.head()
del train
gc.collect()

In [4]:
tempTest = test[['customer_ID','S_2','B_10']]
tempTest.head()
del test
gc.collect()

In [5]:
def dataTransform(df):
    df['S_2'] = pd.to_datetime(df['S_2'])
    df['Year'] = df['S_2'].apply(lambda x:x.year)
    df['Month'] = df['S_2'].apply(lambda x:x.month)
    df['Day'] = df['S_2'].apply(lambda x:x.day)
    df['Year_Month'] = df['Year'].astype(str) + '-' + df['Month'].astype(str)
#     df['Year_Month'] = df['Year_Month'].apply(lambda x: hash(x)%13)

dataTransform(tempTrain)
dataTransform(tempTest)

In [6]:
gc.collect()

In [7]:
tempTrain.info()

In [8]:
tempTrain.head()

In [9]:
tempTrain.isnull().sum()

In [10]:
def plot_plot(df,col):
    sns.distplot(df[col])
    plt.show()
    sns.barplot(df[col])
    plt.show()
    lower = df[col].quantile(0.01)
    upper = df[col].quantile(0.99)
    data = df[(df[col]>lower)&(df[col]<upper)]
    sns.distplot(data[data['target']==1][col],color='blue')
    sns.distplot(data[data['target']==0][col],color='orange')
    plt.show()

plot_plot(tempTrain,'B_10')

In [11]:
tempTrain[tempTrain['B_10']>(tempTrain['B_10'].quantile(0.99))]['target'].value_counts()

In [12]:
tempTrain[tempTrain['B_10']<=(tempTrain['B_10'].quantile(0.99))]['target'].value_counts()

In [13]:
def plot_year_month(df,col):
    plt.figure(figsize=(12,6))
    df['Year_Month'] = pd.to_datetime(df['Year_Month']).dt.date
    data = df.groupby('Year_Month')[col].agg(['mean','median']).reset_index()
    sns.barplot(x='Year_Month', y='mean', data=data)
    plt.show()
    plt.figure(figsize=(12,6))
    sns.barplot(x='Year_Month', y='median', data=data)

plot_year_month(tempTrain,'B_10')

In [14]:
plot_year_month(tempTrain[tempTrain['target']==0],'B_10')

In [15]:
plot_year_month(tempTrain[tempTrain['target']==1],'B_10')

In [16]:
import random
from collections import Counter
def plot_time_feature(df,col,default):
    randCust = df[df['target']==default]['customer_ID'].unique()[random.randint(1,1000)]
    data = df[df['customer_ID']==randCust]
    values = list(data[col].astype('float32'))
    print(values)
    print(Counter(data[col]))
    print('Target',data['target'].iloc[0])
    sns.lineplot(data = values)

plot_time_feature(tempTrain,'B_10',1)


In [17]:
plot_time_feature(tempTrain,'B_10',1)

In [18]:
plot_time_feature(tempTrain,'B_10',0)

In [19]:
plot_time_feature(tempTrain,'B_10',0)

In [20]:
# extract time feature/groupby logic
# provided apis below

# give more weights to the latest months
def time_linear_weight(df):
    _ = list(df)
    weights = [(idx+1)/len(_)*value for idx,value in enumerate(_)]
    return sum(weights)

# you can also define non-linear weight function here
# def time_nonlinearfunc(df):
#     _ = list(df)
#     weights = [...]
#     return ...

# how status changes
def time_trend(df):
    _ = list(df)
    n = len(_)
    half = max(1,int(n/2))
    return np.mean(_[-half:])/np.mean(_)

# ratio
def time_betaR(df):
    _ = list(df)
    return (_[-1]-_[0])/_[0]

# difference
def time_betaD(df):
    _ = list(df)
    return _[-1]-_[0]

# range
def time_range(df):
    _ = list(df)
    return max(_)-min(_)

# mean/std
def time_cov(df):
    _ = list(df)
    return np.mean(_)/np.std(_)

dfb10 = tempTrain.groupby('customer_ID')['B_10'].agg(['mean','median','last',time_linear_weight,time_trend,time_betaR,time_betaD,time_range,time_cov])
dfb10_ = tempTest.groupby('customer_ID')['B_10'].agg(['mean','median','last',time_linear_weight,time_trend,time_betaR,time_betaD,time_range,time_cov])

In [21]:
# now next variable
# plot_plot(tempTrain,'B_?')
# plot_year_month(tempTrain,'B_?')
# plot_year_month(tempTrain[tempTrain['target']==0],'B_?')
# plot_year_month(tempTrain[tempTrain['target']==1],'B_?')
# plot_time_feature(tempTrain,'B_?',1)
# plot_time_feature(tempTrain,'B_?',1)
# plot_time_feature(tempTrain,'B_?',0)
# plot_time_feature(tempTrain,'B_?',0)
# dfb10 = tempTrain.groupby('customer_ID')['B_?'].agg(['mean','median','last',?])
# dfb10_ = tempTest.groupby('customer_ID')['B_?'].agg(['mean','median','last',?])

In [22]:
result = pd.concat([dfb10],axis=1)
result_ = pd.concat([dfb10_],axis=1)

In [23]:
def reduce_size(df):
    for i in df.columns:
        df[i] = df[i].astype('float16')

reduce_size(result)
reduce_size(result_)

In [24]:
result.info()

In [25]:
result.to_pickle('shy_train_num1.pkl')
result_.to_pickle('shy_test_num1.pkl')